In [1]:
## Bracketed Paste is disabled to prevent characters after output
## Example: 
# $ echo 'Hi'
# | Hi?2004l

bind 'set enable-bracketed-paste off'

In [2]:
## TEST:  Make sure simple prompt used (e.g., no escapes that might contaminate output)
## PS1="$ "
## TODO: PS1="> "

## NOTE: The Jupyter bash kernel requires that PS1 not be modified as they customize it. 
echo $PS1

[PEXP\[\]ECT_PROMPT>


In [3]:
# Delete all aliases and function
# TODO: Instead start from pristine environment
unalias -a
alias | wc -l
for f in $(typeset -f | egrep '^\w+'); do unset -f $f; done
typeset -f | egrep '^\w+' | wc -l

0
0


In [4]:
## NOTE: For reproducability, the directory name needs to be fixed
## In place of $$, use a psuedo random number (e,g., 1210)
## *** All output from one run to the next needs to be the same ***
# Setting a temp directory for tests
TMP=/tmp/test-json-alias

## NOTE: Source it directly from the ./tests directory.
BIN_DIR=$PWD/..

## temp_dir=$TMP/test-$$
temp_dir=$TMP/test-3214

mkdir -p "$temp_dir"
# TODO: /bin/rm -rvf "$temp_dir"
cd "$temp_dir"
pwd

#ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"

/tmp/test-json-alias/test-3214


In [5]:
# Count aliases proper
alias | wc -l
# Count functions
typeset -f | egrep '^\w+' | wc -l

1
0


In [6]:
source $BIN_DIR/tomohara-aliases.bash

In [7]:
## EXAMPLES FOR TEST 1 - FROM https://metacpan.org/dist/JSON-PP/view/bin/json_pp
perl -e 'print q|{"foo":"HI MOM!","bar":1234567890000000000000000}|' | pp-json -f json -t dumper

{
  bar => "1234567890000000000000000",
  foo => "HI MOM!"
}


In [8]:
# json-pp WORKS SAME AS pp-json
perl -e'print q|{"foo":"あい","bar":1234567890000000000000000}|' | json-pp -f json -t dumper -json_opt pretty,utf8,allow_bignum

{
  bar => bless( {
    sign => "+",
    value => bless( [
      0,
      890000000,
      1234567
    ], 'Math::BigInt::Calc' )
  }, 'Math::BigInt' ),
  foo => "\x{3042}\x{3044}"
}


In [9]:
# note: canonical sorts the keys of hashes (utf8 avoids warning and pretty might be the default
perl -e'print q|{"foo":"あい","bar":1234567890000000000000000}|' | pp-json-sorted -f json -t dumper

{
  bar => "1234567890000000000000000",
  foo => "\x{3042}\x{3044}"
}


In [10]:
## TEST 2 - TEXT MANIPULATION
# Text manipulation
# alias intersection=intersection.perl
# alias difference=intersection.perl -diff
# alias 'line-intersection=intersection.perl -line'
# alias 'line-difference=intersection.perl -diff -line'

In [11]:
pwd

/tmp/test-json-alias/test-3214


In [12]:
## TEST 2 - TEXT MANIPULATION
rm -rf ./* > /dev/null
printf "This is Line A.\n1 2 3 4 5" > file_a.txt
printf "This is Line B.\n1 3 5 7 9" > file_b.txt

In [13]:
intersection ./file_a.txt ./file_b.txt

This
is
Line
1
3
5


In [14]:
line-intersection ./file_b.txt ./file_a.txt 

In [15]:
difference ./file_a.txt ./file_b.txt

2
4
A.


In [16]:
line-difference ./file_b.txt ./file_a.txt 

1 3 5 7 9
This is Line B.


In [17]:
# THIS IS THE DEFAULT PRINT STATEMENT WITH /r (RETURN)
printf "Hi Mom, \rI am using Ubuntu\n"

I am using Ubuntu


In [18]:
# remove-cr REMOVES RETURNS (or /r)
printf "Hi Mom,\r I am using Ubuntu\n" | remove-cr

Hi Mom, I am using Ubuntu


In [19]:
# alt-remove-cr IS AN ALTERNATIVE OF remove-cr
printf "Hi Mom,\r I am using Ubuntu\n" | alt-remove-cr

Hi Mom, I am using Ubuntu


In [20]:
# perl-remove-cr ACTS AS remove-cr USING PERL
printf "Hi Mom,\r I am using Ubuntu\n" | perl-remove-cr


-i used with no filenames on the command line, reading from STDIN.
Hi Mom, I am using Ubuntu


In [21]:
pwd
rm -rf ./* > /dev/null
printf "Hi Mom,\nThs is the use of printf\nI can use a backslash n to start a new line.\n1\n2\n3"> abc-test.txt
last-n-with-header 3 abc-test.txt

/tmp/test-json-alias/test-3214
Hi Mom,
1
2
3

In [22]:
show-line 3 abc-test.txt

I can use a backslash n to start a new line.
